<a href="https://colab.research.google.com/github/sruthipsuresh/best-sellers-analysis/blob/main/topicmodeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import re
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import seaborn as sns
! pip install -U textblob
from textblob import TextBlob
from google.colab import files
import matplotlib.pyplot as plt
% matplotlib inline


Requirement already up-to-date: textblob in /usr/local/lib/python3.7/dist-packages (0.15.3)


https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21


In [17]:
# Load 
nyt = pd.read_csv("/content/nytcleaned.csv") #loading data
nyt.head()
nyt['results__books__title'] = nyt['results__books__title'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
nyt['results__books__title'] = nyt['results__books__title'].map(lambda x: x.lower())
nyt['results__books__description'] = nyt['results__books__description'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
nyt['results__books__description'] = nyt['results__books__description'].map(lambda x: x.lower())
titlemodel = nyt['results__books__title']
titlemodel.to_csv('titlemodel.csv')

<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:7: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:7: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:7: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:7: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:7: DeprecationWarning: invalid escape sequence \.
<ipython-input-17-31dce65ab50d>:4: DeprecationWarning: invalid escape sequence \.
  nyt['results__books__title'] = nyt['results__books__title'].map(lambda x: re.sub('[,\.!?]', '', x))
<ipython-input-17-31dce65ab50d>:7: DeprecationWarning: invalid

In [18]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')

nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.extend(['from', 'you', 'yall'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = nyt.results__books__title.values.tolist() # switch based on what is needed
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['happened']
[(0, 1)]


In [20]:
from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


[(0,
  '0.262*"mart" + 0.262*"crying" + 0.024*"happened" + 0.024*"caste" + '
  '0.024*"untamed" + 0.024*"nomadland" + 0.024*"antitrust" + '
  '0.024*"greenlights" + 0.024*"think" + 0.024*"thing"'),
 (1,
  '0.262*"breaker" + 0.262*"code" + 0.024*"untamed" + 0.024*"caste" + '
  '0.024*"happened" + 0.024*"greenlights" + 0.024*"nomadland" + '
  '0.024*"antitrust" + 0.024*"think" + 0.024*"pain"'),
 (2,
  '0.262*"many" + 0.262*"one" + 0.024*"happened" + 0.024*"think" + '
  '0.024*"caste" + 0.024*"nomadland" + 0.024*"antitrust" + 0.024*"untamed" + '
  '0.024*"greenlights" + 0.024*"mart"'),
 (3,
  '0.344*"nomadland" + 0.031*"untamed" + 0.031*"happened" + 0.031*"think" + '
  '0.031*"greenlights" + 0.031*"caste" + 0.031*"antitrust" + 0.031*"code" + '
  '0.031*"thing" + 0.031*"best"'),
 (4,
  '0.262*"greenlights" + 0.262*"antitrust" + 0.024*"happened" + '
  '0.024*"untamed" + 0.024*"caste" + 0.024*"nomadland" + 0.024*"think" + '
  '0.024*"pain" + 0.024*"thing" + 0.024*"mafia"'),
 (5,
  '0.262*"be

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [14]:
!pip install gensim
!pip install pyLDAvis


https://colab.research.google.com/github/grantmlong/itds2018/blob/master/lecture-12/DataDive-Lecture12.ipynb#scrollTo=GDsybaWp7-xH

In [21]:
from gensim import corpora
from gensim.models.ldamodel import LdaModel

import pyLDAvis.gensim_models
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
lda_display = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, sort_topics=False)
pyLDAvis.display(lda_display)

pyLDAvis.save_html(lda_display, 'nytonly_title.html')
